## Download the pre-processed data in previous data preparation notebook

In [ ]:
# Download Ground Truth
!mkdir Dataset
!wget  Change this link  -O Dataset/GT.tif

In [ ]:
#import libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import keras
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm
keras.backend.set_image_data_format('channels_last')

In [ ]:
#import and divide dataset
Xdata = np.load("Dataset/Xdata.npy")
Ydata = np.load("Dataset/Ydata.npy")
print(f"the shape of input image matrix is {Xdata.shape}")

In [ ]:
n = 385 #sample number
fig, ax = plt.subplots(1,2, figsize=(12, 6))
ax[0].imshow(Xdata[n,:,:,:3].transpose((0, 1, 2)))
ax[1].imshow(Ydata[n,:,:,0])

ax[0].ticklabel_format(useOffset=False, style='plain')
ax[1].ticklabel_format(useOffset=False, style='plain')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    Xdata, Ydata, test_size=0.3, random_state=42)

### Define model and Train

In [ ]:
model = sm.Unet('resnet34', classes=1, activation='sigmoid')
#https://segmentation-models.readthedocs.io/en/latest/tutorial.html
Unet(backbone_name='resnet34',classes=1, activation='sigmoid', encoder_weights=None, input_shape=(256, 256, 9))

In [ ]:
# Configure the model for training.
# We use the "sparse" version of categorical_crossentropy
# because our target data is integers.
model.compile(
    optimizer=keras.optimizers.Adam(1e-4), loss='binary_crossentropy', metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.MeanIoU(num_classes=2),tf.keras.metrics.Precision(),tf.keras.metrics.Recall()]
)

def trainmodel(model,xdata,ydata):
    
    NUMBER_EPOCHS = 100
    filepath='checkpointMaping'
    BATCH_SIZE=32
    
    model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
        filepath,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        mode="min",
        save_freq="epoch",
        options=None
    )
    print(type(xdata),type(ydata))
    hist = model.fit(x=xdata,
                     y=ydata,
                     epochs=NUMBER_EPOCHS,
                     batch_size=BATCH_SIZE,
                     validation_split=0.2,#auto validate using 20% of random samples at each epoch
                     verbose=1, callbacks=[model_checkpoint_callback],class_weight = {0: 1, 1: 5}

                    )
    return hist
